In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(stringr))
library(ggrepel)
library(reshape2)
library(ggsci)
library(rstatix)
library(ggpubr)
library(hise)


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




# Get DEG Number

In [2]:
DEG_res<-read.csv("/home//jupyter/BRI_Figures_Final_V2/Figure2/02_DEG/Deseq2_Result_Y1D0.csv")

In [3]:
DEG_res_signi<-DEG_res %>% filter(padj<0.05,abs(log2FoldChange)>0.1)

In [4]:
DEG_counts<-as.data.frame(table(DEG_res_signi$contrast,DEG_res_signi$celltype))

In [5]:
colnames(DEG_counts)<-c("factor","AIFI_L3","DEG_Counts")

# Get Freq

In [6]:
BRI_Freq_Data<-read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [7]:
BRI_Freq_Data<-BRI_Freq_Data %>% group_by(sample.sampleKitGuid) %>%
  dplyr::mutate(percentage = (AIFI_L3_count / sum(AIFI_L3_count)) * 100) %>% ungroup()

In [8]:
BRI_Freq_Data_subset<-BRI_Freq_Data %>% filter(sample.visitName=="Flu Year 1 Day 0")

### AgeGroup

In [9]:
#compute freq changes
freq_changes<-BRI_Freq_Data_subset %>%
  select(percentage, AIFI_L3, cohort.cohortGuid, subject.biologicalSex, subject.cmv) %>%
  group_by(AIFI_L3, cohort.cohortGuid) %>%
  summarise(mean_AIFI_L3_clr = median(percentage, na.rm = TRUE)) %>%
  spread(cohort.cohortGuid, mean_AIFI_L3_clr) %>%
  mutate(log2fc = log2(`BR2` / `BR1`)) %>%
  mutate(delta_change = `BR2` - `BR1`) 
#compute freq changes p values

freq_changes_STAT<-BRI_Freq_Data_subset %>%
  select(AIFI_L3_clr,AIFI_L3, cohort.cohortGuid,subject.biologicalSex,subject.cmv) %>%
  group_by(AIFI_L3)  %>%
  wilcox_test(AIFI_L3_clr ~ cohort.cohortGuid) %>%
  adjust_pvalue(method='BH') %>%
  add_significance() %>% as.data.frame()

df<-left_join(freq_changes_STAT,freq_changes)

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.
Joining with `by = join_by(AIFI_L3)`


In [10]:
df<-left_join(df,DEG_counts %>% filter(factor=="cohort.cohortGuid"))

Joining with `by = join_by(AIFI_L3)`


In [15]:
df<-df%>%
  mutate(adjP_degs_combined_value = if_else(DEG_Counts != 0,
                                            -log10(p.adj)  * DEG_Counts ,
                                            -log10(p.adj)  ))

In [16]:
write.csv(df,'FreqChanges_and_DEG.csv')

In [1]:
res<-read.csv('FreqChanges_and_DEG.csv')

In [2]:
res

X,AIFI_L3,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,BR1,BR2,log2fc,delta_change,factor,DEG_Counts,adjP_degs_combined_value
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>
1,ASDC,AIFI_L3_clr,BR1,BR2,47,44,914,3.44e-01,4.943208e-01,ns,0.027138883,0.027744871,0.03185976,0.0006059882,cohort.cohortGuid,0,3.059912e-01
2,Activated memory B cell,AIFI_L3_clr,BR1,BR2,38,37,777,4.38e-01,5.758889e-01,ns,0.019668624,0.016283993,-0.27244157,-0.0033846315,cohort.cohortGuid,0,2.396613e-01
3,Adaptive NK cell,AIFI_L3_clr,BR1,BR2,47,45,793,3.88e-02,1.078654e-01,ns,0.093196645,0.280127431,1.58773334,0.1869307856,cohort.cohortGuid,0,9.671179e-01
4,BaEoMaP cell,AIFI_L3_clr,BR1,BR2,27,19,160,3.12e-02,9.437083e-02,ns,0.005984082,0.008818731,0.55944104,0.0028346486,cohort.cohortGuid,0,1.025162e+00
5,C1Q+ CD16 monocyte,AIFI_L3_clr,BR1,BR2,47,45,778,2.88e-02,9.437083e-02,ns,0.175810538,0.233844493,0.41152791,0.0580339551,cohort.cohortGuid,0,1.025162e+00
6,CD14+ cDC2,AIFI_L3_clr,BR1,BR2,47,45,893,2.01e-01,3.755526e-01,ns,0.356415479,0.361567970,0.02070689,0.0051524918,cohort.cohortGuid,0,4.253292e-01
7,CD27+ effector B cell,AIFI_L3_clr,BR1,BR2,47,45,1149,4.79e-01,5.786500e-01,ns,0.283491164,0.249202552,-0.18598493,-0.0342886117,cohort.cohortGuid,3,7.127521e-01
8,CD27- effector B cell,AIFI_L3_clr,BR1,BR2,47,45,939,3.58e-01,4.943208e-01,ns,0.165279499,0.186751620,0.17621297,0.0214721209,cohort.cohortGuid,2,6.119823e-01
9,CD4 MAIT,AIFI_L3_clr,BR1,BR2,42,42,918,7.52e-01,8.089697e-01,ns,0.039177465,0.036763517,-0.09174928,-0.0024139485,cohort.cohortGuid,0,9.206775e-02
